# Project

# Imports

In [14]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt  # Corrected spelling
import seaborn as sns  # For enhanced data visualization
from scipy import stats  # For statistical tests
import warnings  # To suppress warnings during EDA
import re
warnings.filterwarnings("ignore")  # Optional, to make output cleaner

## Load Data

In [15]:
df_raw = pd.read_excel("data_for_analysis.xlsx")
df_raw.head(3)

,key,fields.issuelinks,fields.Analysis,fields.Target end,fields.Target start,fields.subtasks,fields.votes.votes,fields.issuetype.name,fields.Development,fields.Detected In Build,...,fields.Root Cause Type .value,fields.Root Cause Type .child.value,fields.Escaped Defect Cause Type .value,fields.Should Have Been Found In .value,fields.Escaped Defect Cause Type .child.value,fields.Introduced In .releaseDate,key_y,fields.comment.maxResults,fields.Status .self,fields.Status .value
0,NCCFM-10561,[],NaN,NaN,NaN,[],0,Issue,{summaryBean=com.atlassian.jira.plugin.devstat...,CHF_24_7_I171,...,NaN,NaN,NaN,NaN,NaN,NaN,NCCFM-10561,1,NaN,NaN
1,NCCFM-10560,[],NaN,NaN,NaN,[],0,Issue,{summaryBean=com.atlassian.jira.plugin.devstat...,SPS_24_3_I1671,...,NaN,NaN,NaN,NaN,NaN,NaN,NCCFM-10560,1,NaN,NaN
2,NCCFM-10559,[],NaN,NaN,NaN,[],0,Issue,{summaryBean=com.atlassian.jira.plugin.devstat...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NCCFM-10559,0,NaN,NaN


In [28]:
df = pd.DataFrame({})

## Convert numeric columns

In [29]:
df['numeric_key'] = df_raw['key'].str.split('-').str[1].astype(int)
df['fields.timeestimate'] = df_raw['fields.timeestimate']
df['fields.watches.watchCount'] = pd.to_numeric(df_raw['fields.watches.watchCount'], errors='coerce')



In [30]:
# Function to extract numeric part from CUSTOMER key
def extract_customer_number(customer):
    if pd.notnull(customer) and isinstance(customer, str):  # Check for valid non-blank string
        match = re.search(r'CUSTOMER-(\d+)', customer)  # Match 'CUSTOMER-' followed by numbers
        return int(match.group(1)) if match else None  # Return the number if matched
    return None  # Return None for blanks or missing values

# Extract customer_id and ensure the column exists
df['customers_id'] = df_raw['fields.Customer/s'].apply(extract_customer_number)

# Safely handle missing values
df['customers_id'] = df['customers_id'].fillna(0).astype(int)


In [31]:
# Function to extract and convert the numeric value to int
def extract_customer_id(customer):
    if pd.notnull(customer) and isinstance(customer, str):  # Check if not null and is string
        customer = customer.strip("[]'")  # Remove brackets and quotes
        if customer.isdigit():  # Check if the remaining string is a number
            return int(customer)
    return 0  # Default to 0 if value is invalid or missing

# Apply the function to the column
df['customer_id'] = df_raw['fields.CUSTOMER'].apply(extract_customer_id)

In [32]:
# # Function to extract numeric part from outwardIssue or inwardIssue
# def extract_nccfm_key(issue_links):
#     if issue_links and isinstance(issue_links, list):  # Check if not None and is a list
#         for link in issue_links:
#             # Check outwardIssue
#             if 'outwardIssue' in link and 'key' in link['outwardIssue']:
#                 match = re.search(r'NCCFM-(\d+)', link['outwardIssue']['key'])
#                 if match:
#                     return int(match.group(1))  # Extract numeric part
            
#             # Check inwardIssue
#             if 'inwardIssue' in link and 'key' in link['inwardIssue']:
#                 match = re.search(r'NCCFM-(\d+)', link['inwardIssue']['key'])
#                 if match:
#                     return int(match.group(1))  # Extract numeric part
#     return None  # Return None if no match

# # Load the data
# df_raw = pd.read_excel("data_for_analysis.xlsx")

# # Apply the function to extract the numeric keys
# df_raw['extracted_key'] = df_raw['fields.issuelinks'].apply(extract_nccfm_key)

In [33]:
df.head(18)

,numeric_key,fields.timeestimate,fields.watches.watchCount,customers_id,customer_id
0,10561,NaN,2,52391,0
1,10560,NaN,2,52244,0
2,10559,NaN,2,57634,0
3,10558,NaN,7,77815,0
4,10557,NaN,1,0,0
5,10556,NaN,1,0,0
6,10555,NaN,1,0,0
7,10554,0.0,2,0,0
8,10553,0.0,9,98503,0
9,10552,NaN,1,52391,0


## Convert str columns

In [34]:
df['fields.Analysis'] = df_raw['fields.Analysis'].astype(str)
df['fields.subtasks'] = df_raw['fields.subtasks'].astype(str)
df['fields.description'] = df_raw['fields.description'].astype(str)


## Convert datetime columns

In [35]:
df['fields.Target end'] = pd.to_datetime(df_raw['fields.Target end'], errors='coerce')
df['fields.Target start'] = pd.to_datetime(df_raw['fields.Target start'], errors='coerce')
df['fields.duedate'] = pd.to_datetime(df_raw['fields.duedate'], errors='coerce')
df['fields.resolutiondate'] = pd.to_datetime(df_raw['fields.resolutiondate'], utc=True)
df['fields.Expected Delivery Date'] = pd.to_datetime(df_raw['fields.Expected Delivery Date'], errors='coerce')
df['fields.updated'] = pd.to_datetime(df_raw['fields.updated'], utc=True, errors='coerce')


In [36]:
# Function to split year and PI
def split_year_pi(value):
    if pd.notnull(value) and isinstance(value, str):  # Ensure valid non-null strings
        match = re.search(r"(\d{4})PI(\d+)", value)  # Regex to match year and PI number
        if match:
            return int(match.group(1)), int(match.group(2))  # Return year and PI as integers
    return None, None  # Default for invalid or missing values

# Apply the function to split Year and PI
df_raw[['Year', 'PI']] = df_raw['fields.Program Increment/s'].apply(lambda x: pd.Series(split_year_pi(x)))

# Convert 'Year' to datetime format (first day of the year)
df_raw['Year'] = pd.to_datetime(df_raw['Year'], format='%Y', errors='coerce')

# Convert 'PI' to integer, filling missing values with 0
df_raw['PI'] = df_raw['PI'].fillna(0).astype(int)

In [37]:
df.head(16)

,numeric_key,fields.timeestimate,fields.watches.watchCount,customers_id,customer_id,fields.Analysis,fields.subtasks,fields.description,fields.Target end,fields.Target start,fields.duedate,fields.resolutiondate,fields.Expected Delivery Date,fields.updated
0,10561,NaN,2,52391,0,nan,[],ISU testing from 24.3I189 to 24.7I171.There ar...,NaT,NaT,NaT,NaT,NaT,2024-10-29 03:58:50+00:00
1,10560,NaN,2,52244,0,nan,[],Type of Issue : ME * Problem Statement: Announ...,NaT,NaT,NaT,NaT,NaT,2024-10-28 20:39:46+00:00
2,10559,NaN,2,57634,0,nan,[],Type of Issue: Analysis Problem Statement: Las...,NaT,NaT,NaT,NaT,NaT,2024-10-29 03:10:21+00:00
3,10558,NaN,7,77815,0,nan,[],Type of Issue : < ME > * *Problem Statement:*W...,NaT,NaT,NaT,NaT,NaT,2024-10-28 12:50:49+00:00
4,10557,NaN,1,0,0,nan,[],Type of Issue : < ME / EME / RCA / Analysis > ...,NaT,NaT,NaT,NaT,NaT,2024-10-28 15:21:51+00:00
5,10556,NaN,1,0,0,nan,[],Problem Statement: Daily service polling Inter...,NaT,NaT,NaT,NaT,2024-10-29,2024-10-28 10:49:50+00:00
6,10555,NaN,1,0,0,nan,[],Type of Issue : < ME / EME / RCA / Analysis > ...,NaT,NaT,NaT,NaT,NaT,2024-10-28 08:58:42+00:00
7,10554,0.0,2,0,0,nan,[],Type of Issue : < ME / EME / RCA / Analysis >*...,NaT,NaT,2024-10-27,NaT,NaT,2024-10-27 14:11:08+00:00
8,10553,0.0,9,98503,0,"It is reproducing local lab , There is no func...",[],{*}Issue: -{*}This issue is observed while rem...,NaT,NaT,NaT,2024-10-29 07:33:10+00:00,2024-10-30,2024-10-29 07:33:10+00:00
9,10552,NaN,1,52391,0,nan,[],NOTE: Running helm3 uninstall [sm-csdc] in [na...,NaT,NaT,NaT,NaT,NaT,2024-10-25 19:13:47+00:00


## Convert Categorical columns

In [38]:
df['fields.issuetype.name'] = df_raw['fields.issuetype.name'].astype('category')
df['fields.Priority. .value'] = df_raw['fields.Priority. .value'].astype('category')
df['fields.Origin .value'] = df_raw['fields.Origin .value'].astype('category')



In [39]:
df.head()

,numeric_key,fields.timeestimate,fields.watches.watchCount,customers_id,customer_id,fields.Analysis,fields.subtasks,fields.description,fields.Target end,fields.Target start,fields.duedate,fields.resolutiondate,fields.Expected Delivery Date,fields.updated,fields.issuetype.name,fields.Priority. .value,fields.Origin .value
0,10561,NaN,2,52391,0,nan,[],ISU testing from 24.3I189 to 24.7I171.There ar...,NaT,NaT,NaT,NaT,NaT,2024-10-29 03:58:50+00:00,Issue,Major,Services
1,10560,NaN,2,52244,0,nan,[],Type of Issue : ME * Problem Statement: Announ...,NaT,NaT,NaT,NaT,NaT,2024-10-28 20:39:46+00:00,Issue,Major,Services
2,10559,NaN,2,57634,0,nan,[],Type of Issue: Analysis Problem Statement: Las...,NaT,NaT,NaT,NaT,NaT,2024-10-29 03:10:21+00:00,Issue,Major,Services
3,10558,NaN,7,77815,0,nan,[],Type of Issue : < ME > * *Problem Statement:*W...,NaT,NaT,NaT,NaT,NaT,2024-10-28 12:50:49+00:00,Issue,Major,Services
4,10557,NaN,1,0,0,nan,[],Type of Issue : < ME / EME / RCA / Analysis > ...,NaT,NaT,NaT,NaT,NaT,2024-10-28 15:21:51+00:00,Issue,Major,R&D


## Convert Oridinal columns
